In [36]:
from luminoso_api import LuminosoClient
from voting_classifier.util import train_classifier, classify_documents
from run_voting_classifier import split_train_test, get_all_docs

In [37]:
client = LuminosoClient.connect()

In [38]:
proj_list = client.get()

In [19]:
train_list = []
project_list = []
for i in range(len(proj_list)):
    if "Supercell training" in proj_list[i]['name']:
        train_list.append((proj_list[i]['name'], proj_list[i]['project_id']))
    if "Supercell Finished Validation" in proj_list[i]['name']:
        project_list.append((proj_list[i]['name'], proj_list[i]['project_id']))

In [27]:
game_list = ['CC_', 'CR_']
lang_list = ['AR', 'DE', 'EN', 'ES', 'FR', 'IT', 'PT', 'RU']

In [31]:
# MERGE TRAINING AND TRUE VALIDATION SETS

for i in range(0, len(train_list)):
    client = LuminosoClient.connect()
    training_docs = []
    testing_docs = []
    client = client.change_path(train_list[i][1])
    train_docs, train_labels = get_all_docs(client, 'Label')
    for j in range(len(train_docs)):
        training_docs.append({'title': train_docs[j]['title'], 'text': train_docs[j]['text']})
    
    client = LuminosoClient.connect()
    client = client.change_path(project_list[i][1])
    full_test_docs, full_test_labels = get_all_docs(client, 'Label')
    true_train_docs, test_docs, true_train_labels, test_labels = split_train_test(full_test_docs, full_test_labels)
    for j in range(len(true_train_docs)):
        training_docs.append({'title': true_train_docs[j]['title'], 'text': true_train_docs[j]['text'], 'subsets': 'Label: ' + true_train_labels[j]})
    for j in range(len(test_docs)):
        testing_docs.append({'title': test_docs[j]['title'], 'text': test_docs[j]['text'], 'subsets': 'Label: ' + test_labels[j]})
    
    client = LuminosoClient.connect()
    
    game_name = game_list[int(i / 8)]
    lang_name = lang_list[i % 8]
    proj_id = client.post(name='Supercell hybrid training({})'.format(game_name + lang_name))['project_id']
    client = client.change_path(proj_id)
    client.upload('docs', training_docs)
    client.post('docs/recalculate', language = lang_name.lower())
    client = LuminosoClient.connect()
    proj_id = client.post(name='Supercell hybrid testing({})'.format(game_name + lang_name))['project_id']
    client = client.change_path(proj_id)
    client.upload('docs', testing_docs)
    client.post('docs/recalculate', language = lang_name.lower())
    


In [46]:
# CLASSIFY WITH NEW HYBRID TRAINING

client = LuminosoClient.connect()

account_id = 'd53m338v'
proj_info_list = client.get()
hybrid_training_list = []
hybrid_testing_list = []
for project_info in proj_info_list:
    if 'Supercell hybrid training' in project_info['name']:
        hybrid_training_list.append((project_info['name'], project_info['project_id']))
    if 'Supercell hybrid testing' in project_info['name']:
        hybrid_testing_list.append((project_info['name'], project_info['project_id']))

for i in range(2):
    for j in range(8):
        training_id = hybrid_training_list[(i * 8) + j][1]
        testing_id = hybrid_testing_list[(i * 8) + j][1]
        pickle = '~/Documents/Classifiers/SuperCell/Hybrid_Testing/' + game_list[i] + lang_list[j] + '/'
        %run run_voting_classifier.py $account_id $training_id $testing_id 'Label' Other -f -m -p $pickle -s

Loading Testing & Training documents...
Loaded classifier from /Users/kwang/Documents/Classifiers/SuperCell/Hybrid_Testing/CC_AR/.
Testing classifier...
Saving results to Supercell hybrid testing(CC_AR)_percentage.csv file...
Test Accuracy:84.83%
Loading Testing & Training documents...
Loaded classifier from /Users/kwang/Documents/Classifiers/SuperCell/Hybrid_Testing/CC_DE/.
Testing classifier...
Saving results to Supercell hybrid testing(CC_DE)_percentage.csv file...
Test Accuracy:88.80%
Loading Testing & Training documents...
Loaded classifier from /Users/kwang/Documents/Classifiers/SuperCell/Hybrid_Testing/CC_EN/.
Testing classifier...
Saving results to Supercell hybrid testing(CC_EN)_percentage.csv file...
Test Accuracy:89.59%
Loading Testing & Training documents...
Loaded classifier from /Users/kwang/Documents/Classifiers/SuperCell/Hybrid_Testing/CC_ES/.
Testing classifier...
Saving results to Supercell hybrid testing(CC_ES)_percentage.csv file...
Test Accuracy:86.77%
Loading Test

In [39]:
train_list

[('Supercell training(CC_AR)', 'pr8svf3q'),
 ('Supercell training(CC_DE)', 'pr54w7dq'),
 ('Supercell training(CC_EN)', 'prjqpsvf'),
 ('Supercell training(CC_ES)', 'prsv3th8'),
 ('Supercell training(CC_FR)', 'prt9xhd5'),
 ('Supercell training(CC_IT)', 'pr9bg486'),
 ('Supercell training(CC_PT)', 'prw79mf8'),
 ('Supercell training(CC_RU)', 'prv69kd2'),
 ('Supercell training(CR_AR)', 'prfsq8td'),
 ('Supercell training(CR_DE)', 'prxmk5q7'),
 ('Supercell training(CR_EN)', 'pr2gwt3j'),
 ('Supercell training(CR_ES)', 'prgmkwc7'),
 ('Supercell training(CR_FR)', 'prbnqws5'),
 ('Supercell training(CR_IT)', 'prtr6fgp'),
 ('Supercell training(CR_PT)', 'pr6sv783'),
 ('Supercell training(CR_RU)', 'prxkbgvw')]

In [40]:
project_list

[('Supercell Finished Validation (CC_AR)', 'prchdpqt'),
 ('Supercell Finished Validation (CC_DE)', 'pr7ck6hm'),
 ('Supercell Finished Validation (CC_EN)', 'pr5cbgn3'),
 ('Supercell Finished Validation (CC_ES)', 'prr37mck'),
 ('Supercell Finished Validation (CC_FR)', 'prcvz9d7'),
 ('Supercell Finished Validation (CC_IT)', 'prp48q9h'),
 ('Supercell Finished Validation (CC_PT)', 'prhjf96w'),
 ('Supercell Finished Validation (CC_RU)', 'pr72fbp5'),
 ('Supercell Finished Validation (CR_AR)', 'prk2dz39'),
 ('Supercell Finished Validation (CR_DE)', 'pr7w26gb'),
 ('Supercell Finished Validation (CR_EN)', 'pr9wf584'),
 ('Supercell Finished Validation (CR_ES)', 'pr968p5n'),
 ('Supercell Finished Validation (CR_FR)', 'prkv4jsm'),
 ('Supercell Finished Validation (CR_IT)', 'prg9j26k'),
 ('Supercell Finished Validation (CR_PT)', 'prjcxnpr'),
 ('Supercell Finished Validation (CR_RU)', 'prkxj584')]

In [41]:
hybrid_training_list

[('Supercell hybrid training(CC_AR)', 'prbf8d3p'),
 ('Supercell hybrid training(CC_DE)', 'pr76f8qt'),
 ('Supercell hybrid training(CC_EN)', 'prgq3wd9'),
 ('Supercell hybrid training(CC_ES)', 'prhrm7v5'),
 ('Supercell hybrid training(CC_FR)', 'prgr37wv'),
 ('Supercell hybrid training(CC_IT)', 'przhmd3x'),
 ('Supercell hybrid training(CC_PT)', 'prcq24vw'),
 ('Supercell hybrid training(CC_RU)', 'przprst6'),
 ('Supercell hybrid training(CR_AR)', 'prz3sxhk'),
 ('Supercell hybrid training(CR_DE)', 'prksnmv7'),
 ('Supercell hybrid training(CR_EN)', 'pr3c2j7g'),
 ('Supercell hybrid training(CR_ES)', 'pr78bwcg'),
 ('Supercell hybrid training(CR_FR)', 'prmbsn2z'),
 ('Supercell hybrid training(CR_IT)', 'pr32whqp'),
 ('Supercell hybrid training(CR_PT)', 'prn53zxm'),
 ('Supercell hybrid training(CR_RU)', 'prhzrxcg')]

In [42]:
hybrid_testing_list

[('Supercell hybrid testing(CC_AR)', 'przm7j9n'),
 ('Supercell hybrid testing(CC_DE)', 'prrcx7zv'),
 ('Supercell hybrid testing(CC_EN)', 'prfsr9jq'),
 ('Supercell hybrid testing(CC_ES)', 'prn3trd4'),
 ('Supercell hybrid testing(CC_FR)', 'prs8pkqt'),
 ('Supercell hybrid testing(CC_IT)', 'prksd76n'),
 ('Supercell hybrid testing(CC_PT)', 'prgbw9h7'),
 ('Supercell hybrid testing(CC_RU)', 'prg5zh63'),
 ('Supercell hybrid testing(CR_AR)', 'pr853sgr'),
 ('Supercell hybrid testing(CR_DE)', 'prx53fts'),
 ('Supercell hybrid testing(CR_EN)', 'prjpg8tb'),
 ('Supercell hybrid testing(CR_ES)', 'pr9td7hj'),
 ('Supercell hybrid testing(CR_FR)', 'pr35w2v4'),
 ('Supercell hybrid testing(CR_IT)', 'prfwsvm7'),
 ('Supercell hybrid testing(CR_PT)', 'prvfwgpr'),
 ('Supercell hybrid testing(CR_RU)', 'przdt7np')]

In [49]:
train_docs[1]

{'_id': 'uuid-26d58f73c6734e58acb93faf2445250e',
 'assoc_version': 0,
 'fragments': [],
 'language': 'ru',
 'predict': {},
 'source': {'type': 'Unknown'},
 'subsets': ['__all__', 'Label: Other'],
 'terms': [['нехватает|ru', 'NOUN-PROP/T', [4, 13]],
  ['200гемов|ru', 'NOUN-PROP/T', [14, 22]],
  ['аткрытие|ru', 'NOUN-PROP/T', [26, 34]],
  ['магическова|ru', 'NOUN-PROP/T', [35, 46]],
  ['сундук|ru', 'NOUN/T', [47, 54]],
  ['давать|ru', 'VERB/T', [55, 60]],
  ['пажалюста|ru', 'NOUN-PROP/T', [61, 70]],
  ['гемы|ru', 'NOUN-PROP/T', [71, 75]]],
 'text': 'МНЕ НЕХВАТАЕТ 200ГЕМОВ НА АТКРЫТИЕ МАГИЧЕСКОВА СУНДУКА ДАЙТЕ ПАЖАЛЮСТА ГЕМЫ',
 'title': '58833809',
 'vector': 'W-ejBqSAEG9LNAL5VOjCHy8Ph3a9TOC1RQATHM5kDY6nWnBIXBPj45L_-4-SFCWx-XR7MV_CGDiV44b-lfCO9-f8_2zKNG_L6BIa81Ky5cNcCFRUEaTQFb5bLBX1Cym-TB6-MDlD-yB_uvDeeEh9_S4BJy_x6-WrBmiB4vGLgAwaEmEJh1HAq7lgJkmI8gEuTDPB-1kJ7P81gA_c8qBBvPCUa_4eFXdF6OAIVBok_JFCl0BZlBvOGGZ72sBQ-__95CA7nXFqHFkr830_ucBxw-1ZBY_DTrCDHBMJ8upAxOALx9S2B3__aQEEGDqI8lc-n08rhB5R9eJ-Pe

In [51]:
for i in range(len(hybrid_training_list)):
    client = LuminosoClient.connect()
    client = client.change_path(hybrid_training_list[i][1])
    proj_name = client.get()['name']
    hybrid_docs, hybrid_labels = get_all_docs(client, 'Label')
    upload_docs = []
    for j in range(len(hybrid_docs)):
        if hybrid_labels[j] != 'Other':
            upload_docs.append({'title': hybrid_docs[j]['title'], 
                                'text': hybrid_docs[j]['text'], 
                                'subsets': 'Label: ' + hybrid_labels[j]})
    client = LuminosoClient.connect()
    proj_id = client.post(name=proj_name + ' No Other')['project_id']
    client = client.change_path(proj_id)
    client.upload('docs', upload_docs)
    client.post('docs/recalculate', language=lang_list[i%8].lower())

In [59]:
client = LuminosoClient.connect()

account_id = 'd53m338v'
proj_info_list = client.get()
hybrid_no_other_training_list = []
hybrid_testing_list = []
for project_info in proj_info_list:
    if 'No Other' in project_info['name']:
        hybrid_no_other_training_list.append((project_info['name'], project_info['project_id']))
    if 'Supercell hybrid testing' in project_info['name']:
        hybrid_testing_list.append((project_info['name'], project_info['project_id']))

for i in range(2):
    for j in range(8):
        training_id = hybrid_no_other_training_list[(i * 8) + j][1]
        testing_id = hybrid_testing_list[(i * 8) + j][1]
        pickle = '~/Documents/Classifiers/SuperCell/Hybrid_No_Other_Testing/' + game_list[i] + lang_list[j] + '/'
        %run run_voting_classifier.py $account_id $training_id $testing_id 'Label' Other -f -p $pickle -s -d

Loading Testing & Training documents...
Loaded classifier from /Users/kwang/Documents/Classifiers/SuperCell/Hybrid_No_Other_Testing/CC_AR/.
Testing classifier...
Saving results to Supercell hybrid testing(CC_AR)_percentage.csv file...


ValueError: 'Other' is not in list

Loading Testing & Training documents...
Loaded classifier from /Users/kwang/Documents/Classifiers/SuperCell/Hybrid_No_Other_Testing/CC_DE/.
Testing classifier...
Saving results to Supercell hybrid testing(CC_DE)_percentage.csv file...


ValueError: 'Other' is not in list

Loading Testing & Training documents...
Loaded classifier from /Users/kwang/Documents/Classifiers/SuperCell/Hybrid_No_Other_Testing/CC_EN/.
Testing classifier...
Saving results to Supercell hybrid testing(CC_EN)_percentage.csv file...


...

Loading Testing & Training documents...
No classifier found in /Users/kwang/Documents/Classifiers/SuperCell/Hybrid_No_Other_Testing/CR_IT/.
Training classifier...
Testing classifier...


KeyboardInterrupt: 

Loading Testing & Training documents...


KeyboardInterrupt: 

KeyboardInterrupt: 

In [54]:
hybrid_no_other_training_list

[('Supercell hybrid training(CC_AR) No Other', 'prf392wm'),
 ('Supercell hybrid training(CC_DE) No Other', 'prqst24n'),
 ('Supercell hybrid training(CC_EN) No Other', 'pr2dxqp8'),
 ('Supercell hybrid training(CC_ES) No Other', 'prfgm6ck'),
 ('Supercell hybrid training(CC_FR) No Other', 'prhxn5qv'),
 ('Supercell hybrid training(CC_IT) No Other', 'prkxd9w6'),
 ('Supercell hybrid training(CC_PT) No Other', 'prbc4ftq'),
 ('Supercell hybrid training(CC_RU) No Other', 'pr4sv57d'),
 ('Supercell hybrid training(CR_AR) No Other', 'pr4jgwhr'),
 ('Supercell hybrid training(CR_DE) No Other', 'prnv5qhm'),
 ('Supercell hybrid training(CR_EN) No Other', 'pr5q8dbz'),
 ('Supercell hybrid training(CR_ES) No Other', 'pr35t4qw'),
 ('Supercell hybrid training(CR_FR) No Other', 'przkws7q'),
 ('Supercell hybrid training(CR_IT) No Other', 'prtmkdj2'),
 ('Supercell hybrid training(CR_PT) No Other', 'prj2fvxh'),
 ('Supercell hybrid training(CR_RU) No Other', 'prhgkzf6')]